# Convolutional Neural Network

In [ ]:
!git clone "https://github.com/vadivukar/GV_DeepLearning/"

Cloning into 'GV_DeepLearning'...
remote: Enumerating objects: 1607, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 1607 (delta 4), reused 15 (delta 2), pack-reused 1589 (from 1)
Receiving objects: 100% (1607/1607), 29.99 MiB | 21.66 MiB/s, done.
Resolving deltas: 100% (92/92), done.


In [ ]:

!git clone https://github.com/laxmimerit/dog-cat-full-dataset

Cloning into 'dog-cat-full-dataset'...
remote: Enumerating objects: 25027, done.
remote: Total 25027 (delta 0), reused 0 (delta 0), pack-reused 25027 (from 1)
Receiving objects: 100% (25027/25027), 541.62 MiB | 25.63 MiB/s, done.
Resolving deltas: 100% (5/5), done.
Updating files: 100% (25001/25001), done.


### Importing the libraries

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator  # Import from tensorflow.keras
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization

In [ ]:
tf.__version__

'2.17.0'

## Part 1 - Data Preprocessing

### Preprocessing the Training set

In [ ]:
import pandas as pd

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

training_set = train_datagen.flow_from_directory('/content/dog-cat-full-dataset/data/train',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')


Found 20000 images belonging to 2 classes.


### Preprocessing the Test set

In [ ]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('/content/dog-cat-full-dataset/data/test',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')

Found 5000 images belonging to 2 classes.


## Part 2 - Building the CNN

### Initialising the CNN

In [ ]:
cnn = tf.keras.models.Sequential()

### Step 1 - Convolution

In [ ]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu',
                               input_shape=[64, 64, 3]))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Batch Normalization

In [ ]:
cnn.add(tf.keras.layers.BatchNormalization()) # Use 'cnn' instead of 'model'

### Step 2 - Pooling

In [ ]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Adding a second convolutional layer

In [ ]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Step 3 - Flattening

In [ ]:
cnn.add(tf.keras.layers.Flatten())

### Step 4 - Full Connection

In [ ]:
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

### Step 5 - Output Layer

In [ ]:
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Part 3 - Training the CNN

### Compiling the CNN

In [ ]:
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Training the CNN on the Training set and evaluating it on the Test set

In [ ]:
# Train the model with batch normalization
history_with_bn = cnn.fit(x = training_set, validation_data = test_set, epochs = 2)

Epoch 1/2
625/625 ━━━━━━━━━━━━━━━━━━━━ 207s 331ms/step - accuracy: 0.7643 - loss: 0.4832 - val_accuracy: 0.7826 - val_loss: 0.4620
Epoch 2/2
625/625 ━━━━━━━━━━━━━━━━━━━━ 206s 328ms/step - accuracy: 0.7889 - loss: 0.4475 - val_accuracy: 0.7868 - val_loss: 0.4411


In [ ]:
# Evaluate the model with batch normalization
loss_with_bn, accuracy_with_bn = cnn.evaluate(test_set)
print(f"Test Accuracy with Batch Normalization: {accuracy_with_bn * 100:.2f}%")

157/157 ━━━━━━━━━━━━━━━━━━━━ 15s 92ms/step - accuracy: 0.7975 - loss: 0.4304
Test Accuracy with Batch Normalization: 78.68%


In [ ]:
# Define the model without batch normalization
model_without_bn = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3]),
    tf.keras.layers.MaxPool2D(pool_size=2, strides=2),

    tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'),
    tf.keras.layers.MaxPool2D(pool_size=2, strides=2),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=128, activation='relu'),
    tf.keras.layers.Dense(units=1, activation='sigmoid')
])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
# Compile the model without batch normalization
model_without_bn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])


In [ ]:
# Train the model without batch normalization
history_without_bn = model_without_bn.fit(x = training_set, validation_data = test_set, epochs = 2)

Epoch 1/2
625/625 ━━━━━━━━━━━━━━━━━━━━ 160s 253ms/step - accuracy: 0.5833 - loss: 0.6640 - val_accuracy: 0.6242 - val_loss: 0.6800
Epoch 2/2
625/625 ━━━━━━━━━━━━━━━━━━━━ 200s 250ms/step - accuracy: 0.7092 - loss: 0.5673 - val_accuracy: 0.7294 - val_loss: 0.5568


In [ ]:
# Evaluate the model without batch normalization
loss_without_bn, accuracy_without_bn = model_without_bn.evaluate(test_set)
print(f"Test Accuracy without Batch Normalization: {accuracy_without_bn * 100:.2f}%")

157/157 ━━━━━━━━━━━━━━━━━━━━ 14s 88ms/step - accuracy: 0.7295 - loss: 0.5559
Test Accuracy without Batch Normalization: 72.94%


## Part 4 - Making a single prediction

In [ ]:
cnn.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)                    │ (None, 62, 62, 32)          │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 62, 62, 32)          │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_4 (MaxPooling2D)       │ (None, 31, 31, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_4 (Conv2D)                    │ (None, 29, 29, 32)          │           9,248 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_5 (MaxPooling2D)       │ (None, 14, 14, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 6272)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 128)                 │         802,944 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,439,909 (9.31 MB)

 Trainable params: 813,281 (3.10 MB)

 Non-trainable params: 64 (256.00 B)

 Optimizer params: 1,626,564 (6.20 MB)

In [ ]:
import numpy as np
from keras.preprocessing import image

test_image = tf.keras.utils.load_img('/content/GV_DeepLearning/Dataset/SingleImage/cat_or_dog_1.jpg',
                            target_size = (64, 64))
#test_image = tf.keras.utils.load_img(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = cnn.predict(test_image)
training_set.class_indices
if result[0][0] == 1:
  prediction = 'dog'
else:
  prediction = 'cat'

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 156ms/step


In [ ]:
print(prediction)

dog


In [ ]:
# Compare the accuracies
print(f"Accuracy with Batch Normalization: {accuracy_with_bn * 100:.2f}%")
print(f"Accuracy without Batch Normalization: {accuracy_without_bn * 100:.2f}%")

Accuracy with Batch Normalization: 78.68%
Accuracy without Batch Normalization: 72.94%
